In [1]:
import numpy as np
import tensorflow as tf
import cPickle as pickle
import gzip
import time
import random

In [5]:
with gzip.open('mnielsen/data/mnist.pkl.gz', 'rb') as f:
    train_dataset, valid_dataset, test_dataset = pickle.load(f)

print "Training set", train_dataset[0].shape, train_dataset[1].shape
print "Validation set", valid_dataset[0].shape, valid_dataset[1].shape
print "Test set", test_dataset[0].shape, test_dataset[1].shape    

Training set (50000, 784) (50000,)
Validation set (10000, 784) (10000,)
Test set (10000, 784) (10000,)


In [15]:
expanded_pairs = []
j = 0
for x, y in zip(train_dataset[0], train_dataset[1]):
    expanded_pairs.append((x,y))
    image = np.reshape(x, (-1, 28))
    j += 1
    if j % 1000 == 0: print ("Expanding impage number", j)
    for d, axis, index_position, index in [
        (1, 0, "first", 0),
        (-1, 0, "first", 27),
        (1, 1, "last", 0),
        (-1, 1, "last", 27)]:
        new_img = np.roll(image, d, axis)
        if index_position == 'first':
            new_img[index, :] = np.zeros(28)
        else:
            new_img[:, index] = np.zeros(28)
        expanded_pairs.append((np.reshape(new_img, 784), y))
random.shuffle(expanded_pairs)
expanded_training = [list(d) for d in zip(*expanded_pairs)]
print "Saving expanded data. This may take a few minutes."
f = gzip.open("mnist_expanded.pkl.gz", "w")
pickle.dump((expanded_training, valid_dataset, test_dataset), f)
f.close()
print "Done"

('Expanding impage number', 1000)
('Expanding impage number', 2000)
('Expanding impage number', 3000)
('Expanding impage number', 4000)
('Expanding impage number', 5000)
('Expanding impage number', 6000)
('Expanding impage number', 7000)
('Expanding impage number', 8000)
('Expanding impage number', 9000)
('Expanding impage number', 10000)
('Expanding impage number', 11000)
('Expanding impage number', 12000)
('Expanding impage number', 13000)
('Expanding impage number', 14000)
('Expanding impage number', 15000)
('Expanding impage number', 16000)
('Expanding impage number', 17000)
('Expanding impage number', 18000)
('Expanding impage number', 19000)
('Expanding impage number', 20000)
('Expanding impage number', 21000)
('Expanding impage number', 22000)
('Expanding impage number', 23000)
('Expanding impage number', 24000)
('Expanding impage number', 25000)
('Expanding impage number', 26000)
('Expanding impage number', 27000)
('Expanding impage number', 28000)
('Expanding impage number', 2

In [7]:
with gzip.open('mnist_expanded1.pkl.gz', 'rb') as f:
    train_dataset, valid_dataset, test_dataset = pickle.load(f)

print "Training set", np.array(train_dataset[0]).shape, np.array(train_dataset[1]).shape
print "Validation set", valid_dataset[0].shape, valid_dataset[1].shape
print "Test set", test_dataset[0].shape, test_dataset[1].shape    

Training set (250000, 784) (250000,)
Validation set (10000, 784) (10000,)
Test set (10000, 784) (10000,)


In [8]:
def vectorized_result(j):
    e = np.zeros(10)
    e[j] = 1.0
    return e

train_data = np.array([np.reshape(x, (28,28,1)) for x in train_dataset[0]])
train_labels = np.array([vectorized_result(y) for y in train_dataset[1]])
valid_data = np.array([np.reshape(x, (28,28,1)) for x in valid_dataset[0]])
valid_labels = np.array([vectorized_result(y) for y in valid_dataset[1]])
test_data = np.array([np.reshape(x, (28,28,1)) for x in test_dataset[0]])
test_labels = np.array([vectorized_result(y) for y in test_dataset[1]])
print "Training set", train_data.shape, train_labels.shape
print "Validation set", valid_data.shape, valid_labels.shape
print "Test set", test_data.shape, test_labels.shape

Training set (250000, 28, 28, 1) (250000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [9]:
batch_size = 10

graph = tf.Graph()

with graph.as_default():
    
    # Input data
    ### For convolutional layers
    x = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
    y = tf.placeholder(tf.float32, shape=(None, 10))
    ### For fully connected layers
    # x = tf.placeholder(tf.float32, shape=(None, 784))
    # y = tf.placeholder(tf.float32, shape=(None, 10))
    
    # Model
    W = tf.Variable(tf.truncated_normal([5, 5, 1, 20], stddev=np.sqrt(1.0/(125))))
    b = tf.Variable(tf.truncated_normal([20]))
    a = tf.nn.conv2d(x, W, [1, 1, 1, 1], padding='VALID')
    pool = tf.nn.max_pool(a, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    z = tf.nn.relu(pool + b)
    # For variable-sized batches tf.shape(x)[0]
    # z = tf.reshape(a, [tf.shape(x)[0], (20*12*12)])
    
    W0 = tf.Variable(tf.truncated_normal([5, 5, 20, 40], stddev=np.sqrt(1.0/(250))))
    b0 = tf.Variable(tf.truncated_normal([40]))
    a = tf.nn.conv2d(z, W0, [1, 1, 1, 1], padding='VALID')
    pool = tf.nn.max_pool(a, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    a = tf.nn.relu(pool + b0)
    # For variable-sized batches tf.shape(x)[0]
    z = tf.reshape(a, [tf.shape(x)[0], (40*4*4)])
    
    
    W1 = tf.Variable(tf.truncated_normal([(40*4*4), 100], stddev=np.sqrt(1.0/(100))))
    b1 = tf.Variable(tf.truncated_normal([100]))
    a = tf.nn.relu(tf.matmul(z, W1) + b1)
    
    W2 = tf.Variable(tf.truncated_normal([100, 10], stddev=np.sqrt(1.0/10)))
    b2 = tf.Variable(tf.truncated_normal([10]))
    z = tf.matmul(a, W2) + b2
    # 99.0% 
    
    # Training
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(z, y))
    
    loss += 0.1 * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.03).minimize(loss)
    
    # Predictions
    predictions = tf.nn.softmax(z)
    

In [10]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [11]:
num_epochs = 60
num_steps = 50000/batch_size

'''
for each epoch
   randomly shuffle training data
   for each N size batch of training data
      train model
   view train cost
   compute train accuracy
   view valid cost
   compute valid accuracy
compute test accuracy
'''

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized"
    best_validation_acc = 0.0
    for epoch in range(num_epochs):
        start_time = time.time()
        for step in range(num_steps):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            batch_data = train_data[offset:(offset + batch_size), :, :, :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Feed dictionary
            feed_dict = {x: batch_data, y: batch_labels}
            _, C, pred = session.run([optimizer,loss,predictions],feed_dict=feed_dict)
            if (step % 1000 == 0):
                print("Minibatch loss at step %d: %f" % (step, C))
                print("Minibatch accuracy: %.2f%%" % accuracy(pred, batch_labels))
                feed_dict = {x: valid_data}
                pred = session.run(predictions, feed_dict=feed_dict)
                valid_acc = accuracy(np.array(pred), valid_labels)
                print("Validation accuracy: %.2f%%" % valid_acc)
                if (valid_acc >= best_validation_acc):
                    best_validation_acc = valid_acc
                    print "Best Validation accuracy at step", step, "and epoch", epoch
        print ("Duration: %.2f sec" % (time.time() - start_time))
    feed_dict = {x: test_data}
    pred = session.run(predictions, feed_dict=feed_dict)
    print ("Test accuracy: %.2f%%" % accuracy(np.array(pred), test_labels))



Initialized
Minibatch loss at step 0: 35.163559
Minibatch accuracy: 20.00%
Validation accuracy: 9.67%
Best Validation accuracy at step 0 and epoch 0
Minibatch loss at step 1000: 1.546647
Minibatch accuracy: 50.00%
Validation accuracy: 81.05%
Best Validation accuracy at step 1000 and epoch 0
Minibatch loss at step 2000: 1.220054
Minibatch accuracy: 70.00%
Validation accuracy: 86.78%
Best Validation accuracy at step 2000 and epoch 0
Minibatch loss at step 3000: 0.311473
Minibatch accuracy: 100.00%
Validation accuracy: 94.67%
Best Validation accuracy at step 3000 and epoch 0
Minibatch loss at step 4000: 0.345153
Minibatch accuracy: 100.00%
Validation accuracy: 91.87%
Duration: 19.05 sec
Minibatch loss at step 0: 0.499727
Minibatch accuracy: 90.00%
Validation accuracy: 95.33%
Best Validation accuracy at step 0 and epoch 1
Minibatch loss at step 1000: 0.702190
Minibatch accuracy: 80.00%
Validation accuracy: 86.72%
Minibatch loss at step 2000: 1.022972
Minibatch accuracy: 60.00%
Validation a